In [1]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd

load_dotenv()

api_key = os.environ.get('riot_api_key')

In [2]:
service_account = pygsheets.authorize(
    service_account_file='JSONS\spreadsheet-automator-419713-f0c339d68def.json')

sheet = service_account.open_by_url(
    'https://docs.google.com/spreadsheets/d/1GoBz-6avCyXPqJeO6qz_hxrhPPW_tNPKcIp_G6dKAQ4/edit?usp=sharing')

In [3]:
from urllib import response


def get_puuid(summonerId=None, gameName=None, tagLine=None, region='br1'):
    if summonerId is not None:
        # Pega o puuid pelo summonerId ou pela gameTag+tagLine

        root_url = f'https://{region}.api.riotgames.com'
        endpoint = '/lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+api_key)
        return response.json()['puuid']


def get_idtag_from_puuid(puuid=None, region='br1'):
    # Pega o riot_id e o id_tag pelo puuid

    root_url = f'https://{region}.api.riotgames.com'
    endpoint = 'riot/account/v1/accounts/by-puuid/'
    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id


def get_ladder(top=None, region='br1'):
    root = f'https://{region}.api.riotgames.com'
    chall = '/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = '/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = '/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root+chall+'?api_key='+api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame()
    master_df = pd.DataFrame()

    if top > 300:
        gm_response = requests.get(root+gm+'?api_key='+api_key)
        gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)
    if top > 1000:
        master_response = requests.get(root+master+'?api_key='+api_key)
        master_df = pd.DataFrame(master_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, master_df]).reset_index(drop=True)[:top]
    ladder = ladder.drop(columns='rank').reset_index(
        drop=False).rename(columns={'index': 'rank'})
    ladder['rank'] += 1

    return ladder

def get_match_history(region=None, puuid=None, count=20):
    root = f'https://{region}.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    count = f'?start=0&count={count}&'
    response = requests.get(root+endpoint+count+'&api_key='+api_key)

    return response.json()

def get_match(region=None, matchId=None):
    root = f'https://{region}.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/{matchId}'
    response = requests.get(root+endpoint+'?api_key='+api_key)

    return response.json()

    

In [4]:
get_match_history(region='americas', puuid='nLcOi3gpM4WUybwDjCXHKeW1qDLehxlGOxgRyxpWi9f7HxfAAFEwqlCXFsM-y0J5MqyqsY8y79xg2g')

['BR1_2918466941',
 'BR1_2918259478',
 'BR1_2918216826',
 'BR1_2918181366',
 'BR1_2918156928',
 'BR1_2918122030',
 'BR1_2918082594',
 'BR1_2918071191',
 'BR1_2918041908',
 'BR1_2918029938',
 'BR1_2917801140',
 'BR1_2917762246',
 'BR1_2917740062',
 'BR1_2917709177',
 'BR1_2917660165',
 'BR1_2917628491',
 'BR1_2917605474',
 'BR1_2917576786',
 'BR1_2917556454',
 'BR1_2917435281']

In [5]:
get_match(region='americas', matchId='BR1_2918466941')

{'metadata': {'dataVersion': '2',
  'matchId': 'BR1_2918466941',
  'participants': ['6jiDy-ixfOCUC5yT-9aVPDWYsotl2DmUa6aeMqE5-WZUxHK2SyTsosPHH2nqLjb0x4vyWje-vC_UBQ',
   'HcOIG5oeWJubmiH71qExmxepNd1zW4biakHZUslvr2D749T2ruF0HoBu7DpJ8a4jabkqVrB6DJ31MA',
   '_myqhJ4jLHTOKk5ZhANXRtcMxlvJYXG3NBxuSV_R-csmUJ-vq6kTp8jpVnu96BuqFurLmErlra_Bsw',
   'lDt-nG1Pm97ObS9y1Lod1JRttvpb6fHjKhEKOOfDZkfcCicF09F-AceD5LqXF9HvFISezig1gBXdCA',
   'T7jcg7ck14eP1ir7hn9mTUebytlNGfG7Jsaxrg9iAHFYKByifL6yP44RDsQLr3Z4kMWHwdwPeLe11Q',
   'bQgMDLZgwg5aycq2flU2GWSB7WyQXdKJx_b49HOkmY66aZHcUFA3nivamy5F5t4bhI22AgyCby18ow',
   '4neLdKVjlstCAxMRHwbIj2A4WREDjLPAfWJISaXge4tUiX7ZOsykEky0AhUgDE1CVfeitiriEPB67w',
   'SvEC--CMd0XTg-B-IzAiXPZLk4-fZcRS1lGO5XNXk0C5kTTIFKvhzKZfTRq_nwE6V_RsVyIKqCkePw',
   'kNHrWgn-CZyT0NQNX24eKnSrV05dHk6XB3bCf0lIcIiMUr5IHFls9MiYNocoTe-z3bNEekd9B7HMPg',
   'nLcOi3gpM4WUybwDjCXHKeW1qDLehxlGOxgRyxpWi9f7HxfAAFEwqlCXFsM-y0J5MqyqsY8y79xg2g']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1712

In [6]:
get_puuid(summonerId='g250Cv7oaxE9gb-eysxOwyasvT7kJje3uMDDOJsiVnhqabg')

'nLcOi3gpM4WUybwDjCXHKeW1qDLehxlGOxgRyxpWi9f7HxfAAFEwqlCXFsM-y0J5MqyqsY8y79xg2g'